In [140]:
import pandas as pd
import numpy as np

#!pip install PyMySQL
import pymysql

# Python에서 MySQL 접속 후 테이블 추출

In [141]:
#MySQL Connection 연결
con = pymysql.connect(host='localhost', user='root', password='useruser',
                      db='kpmg', charset='utf8',# 한글처리 (charset = 'utf8')
                      autocommit=True, # 결과 DB 반영 (Insert or update)
                      cursorclass=pymysql.cursors.DictCursor # DB조회시 컬럼명을 동시에 보여줌
) 

In [142]:
# 직급 정보가 담긴 kpmg.position_code 테이블 

cur = con.cursor()
sql_1 = "SELECT * FROM kpmg.position_code"
cur.execute(sql_1)
 
position_code_row = cur.fetchall()
position_code_row = pd.DataFrame(position_code_row)
position_code_row

,position_code,position
0,1000,사장
1,2000,부사장
2,3000,상무
3,4000,이사
4,5000,부장
5,6000,리드
6,7000,팀장
7,8000,대리
8,9000,사원


In [143]:
# 사원 정보가 담긴 kpmg.emp 테이블 

cur = con.cursor()
sql_2 = "SELECT * FROM kpmg.emp"
cur.execute(sql_2)
 
employee_db_row= cur.fetchall()
employee_db_row = pd.DataFrame(employee_db_row)
employee_db_row

,emp_name,emp_phone,position,dept,task
0,김명헌,01011551123,사원,감사B팀,회계감사
1,백지원,01011551123,사원,감사A팀,회계감사
2,서수빈,01075451233,리드,감사A팀,회계감사
3,윤채원,01011121123,대리,감사A팀,회계감사
4,임정민,01011551123,팀장,감사B팀,회계감사
5,최낙현,01011551123,대리,감사B팀,회계감사


In [144]:
# 실시간 catchup 정보가 담긴 kpmg.slack_messages 테이블 

# 추가 테이블 
# STEP 3: Connection 으로부터 Cursor 생성
cur = con.cursor()

# STEP 4: SQL문 실행 및 Fetch
sql_3 = "SELECT * FROM kpmg.slack_messages"
cur.execute(sql_3)
 
# 데이터 Fetch
kpmg_slack_message_row = cur.fetchall()
kpmg_slack_message_row = pd.DataFrame(kpmg_slack_message_row)
kpmg_slack_message_row

,timeline,writer,corp,message
0,2023-02-04 10:02:00,서수빈,이지스밸류플러스리츠,"부속명세서, 세부원장 확인중, 재고실사 해당없음"
1,2023-02-04 10:11:07,윤채원,대신케이리츠물류1호위탁관리부동산투자회사,"세부원장 확인중, 재고실사 해당없음, 금융기관조회서 일치확인, 감사의견 적정"
2,2023-02-04 10:14:54,서수빈,이지스밸류플러스리츠,"부속명세서 및 세부원장 확인완료, 금융기관조회서 일치확인, 감사의견 적정"
3,2023-02-04 10:35:14,임정민,대신정보통신,대신정보통신 담당자 변경 알림: 감사A팀 백지원 → 감사B팀 김명헌
4,2023-02-04 10:57:23,최낙현,JTC,"부속명세서, 세부원장 확인완료, 세부원장 확인완료, 감사의견 적정"
5,2023-02-04 11:12:42,임정민,JTC,감사의견 피드백 : 재검토 요망
6,2023-02-04 13:23:26,임정민,이씨에스,감사증거(회계기록) 추가 확보를 위한 이씨에스 출장 필요
7,2023-02-04 13:32:14,김명헌,이씨에스,당사와 연락 완료 : 2월 5일 오후 2시 출장 예정
8,2023-02-04 14:48:04,백지원,대신정보통신,감사증거(회계기록) 전달 안내
9,2023-02-04 14:50:37,최낙현,JTC,"[re] 부속명세서 및 세부원장 확인완료, 세부원장 확인완료, 감사의견 적정"


In [145]:
# kpmg.emp 테이블의 직급 정보와 kpmg.slack_message 테이블을 연결 

# STEP 3: Connection 으로부터 Cursor 생성
cur = con.cursor()

# STEP 4: SQL문 실행 및 Fetch
sql_select = '''
SELECT kpmg.slack_messages.timeline, kpmg.slack_messages.writer, kpmg.position_code.position_code, kpmg.emp.position, kpmg.slack_messages.corp, kpmg.slack_messages.message
FROM kpmg.emp, kpmg.position_code, kpmg.slack_messages
WHERE kpmg.position_code.position = kpmg.emp.position and kpmg.emp.emp_name = kpmg.slack_messages.writer
'''
cur.execute(sql_select)
 
# 데이터 Fetch
sql_select_row = cur.fetchall()
message_db = pd.DataFrame(sql_select_row)
message_db

,timeline,writer,position_code,position,corp,message
0,2023-02-04 10:02:00,서수빈,6000,리드,이지스밸류플러스리츠,"부속명세서, 세부원장 확인중, 재고실사 해당없음"
1,2023-02-04 10:11:07,윤채원,8000,대리,대신케이리츠물류1호위탁관리부동산투자회사,"세부원장 확인중, 재고실사 해당없음, 금융기관조회서 일치확인, 감사의견 적정"
2,2023-02-04 10:14:54,서수빈,6000,리드,이지스밸류플러스리츠,"부속명세서 및 세부원장 확인완료, 금융기관조회서 일치확인, 감사의견 적정"
3,2023-02-04 10:35:14,임정민,7000,팀장,대신정보통신,대신정보통신 담당자 변경 알림: 감사A팀 백지원 → 감사B팀 김명헌
4,2023-02-04 10:57:23,최낙현,8000,대리,JTC,"부속명세서, 세부원장 확인완료, 세부원장 확인완료, 감사의견 적정"
5,2023-02-04 11:12:42,임정민,7000,팀장,JTC,감사의견 피드백 : 재검토 요망
6,2023-02-04 13:23:26,임정민,7000,팀장,이씨에스,감사증거(회계기록) 추가 확보를 위한 이씨에스 출장 필요
7,2023-02-04 13:32:14,김명헌,9000,사원,이씨에스,당사와 연락 완료 : 2월 5일 오후 2시 출장 예정
8,2023-02-04 14:48:04,백지원,9000,사원,대신정보통신,감사증거(회계기록) 전달 안내
9,2023-02-04 14:50:37,최낙현,8000,대리,JTC,"[re] 부속명세서 및 세부원장 확인완료, 세부원장 확인완료, 감사의견 적정"


In [146]:
# STEP 5: DB 연결 종료
con.close()

# MySQL 연결이 불가능 할 경우

In [147]:
#MySQL 데이터 확인 
position_code_row = pd.read_csv('position_code_row.csv',engine='python',encoding='utf-8',index_col=0)
employee_db_row = pd.read_csv('employee_db_row.csv',engine='python',encoding='utf-8',index_col=0)
kpmg_slack_message_row = pd.read_csv('kpmg_slack_message_row.csv',engine='python',encoding='utf-8',index_col=0)

#catch-up을 위해 select한 데이터 
message_db = pd.read_csv('sql_select_row.csv',engine='python',encoding='utf-8',index_col=0)

# 기업별 catch-up 정보 추출

In [148]:
#sql에서 select한 데이터 

message_db

,timeline,writer,position_code,position,corp,message
0,2023-02-04 10:02:00,서수빈,6000,리드,이지스밸류플러스리츠,"부속명세서, 세부원장 확인중, 재고실사 해당없음"
1,2023-02-04 10:11:07,윤채원,8000,대리,대신케이리츠물류1호위탁관리부동산투자회사,"세부원장 확인중, 재고실사 해당없음, 금융기관조회서 일치확인, 감사의견 적정"
2,2023-02-04 10:14:54,서수빈,6000,리드,이지스밸류플러스리츠,"부속명세서 및 세부원장 확인완료, 금융기관조회서 일치확인, 감사의견 적정"
3,2023-02-04 10:35:14,임정민,7000,팀장,대신정보통신,대신정보통신 담당자 변경 알림: 감사A팀 백지원 → 감사B팀 김명헌
4,2023-02-04 10:57:23,최낙현,8000,대리,JTC,"부속명세서, 세부원장 확인완료, 세부원장 확인완료, 감사의견 적정"
5,2023-02-04 11:12:42,임정민,7000,팀장,JTC,감사의견 피드백 : 재검토 요망
6,2023-02-04 13:23:26,임정민,7000,팀장,이씨에스,감사증거(회계기록) 추가 확보를 위한 이씨에스 출장 필요
7,2023-02-04 13:32:14,김명헌,9000,사원,이씨에스,당사와 연락 완료 : 2월 5일 오후 2시 출장 예정
8,2023-02-04 14:48:04,백지원,9000,사원,대신정보통신,감사증거(회계기록) 전달 안내
9,2023-02-04 14:50:37,최낙현,8000,대리,JTC,"[re] 부속명세서 및 세부원장 확인완료, 세부원장 확인완료, 감사의견 적정"


In [149]:
#회사명 추출 

corp_list = list(set(list(message_db['corp'])))
corp_list

['대신케이리츠물류1호위탁관리부동산투자회사', '이지스밸류플러스리츠', 'JTC', '대신정보통신', '이씨에스']

In [150]:
#catch-up 데이터 추출 후 저장 

catch_up = [ ]

for i in range(len(corp_list)):
    catch_up_db = message_db.sort_values('corp').reset_index(drop=True) #회사명을 기준으로 정렬
    catch_up_db = catch_up_db[catch_up_db['corp'] == corp_list[i] ] #회사 검색
    catch_up_db = catch_up_db.loc[catch_up_db['position_code'].idxmin()] #직급이 높은 직원을 추출  
    catch_up_index = catch_up_db.index 
    catch_up_values = catch_up_db.values
    catch_up.append(catch_up_values) 
    
catch_up_fin = pd.DataFrame(catch_up, columns = catch_up_index)

In [151]:
catch_up_fin

,timeline,writer,position_code,position,corp,message
0,2023-02-04 10:11:07,윤채원,8000,대리,대신케이리츠물류1호위탁관리부동산투자회사,"세부원장 확인중, 재고실사 해당없음, 금융기관조회서 일치확인, 감사의견 적정"
1,2023-02-04 10:02:00,서수빈,6000,리드,이지스밸류플러스리츠,"부속명세서, 세부원장 확인중, 재고실사 해당없음"
2,2023-02-04 11:12:42,임정민,7000,팀장,JTC,감사의견 피드백 : 재검토 요망
3,2023-02-04 10:35:14,임정민,7000,팀장,대신정보통신,대신정보통신 담당자 변경 알림: 감사A팀 백지원 → 감사B팀 김명헌
4,2023-02-04 13:23:26,임정민,7000,팀장,이씨에스,감사증거(회계기록) 추가 확보를 위한 이씨에스 출장 필요


# 기존 기업정보에 catch-up messages 추가

In [153]:
#기존 기업 정보 불러오기

corp_db = pd.read_csv('df_for_chatbot.csv',engine='python',encoding='utf-8', index_col=0)
corp_db["catch_up"] = ""

corp_db

,corp_name,Busines_report,summary,kye_list,classify,manager,recent_progress,last_audit grade,corp_size,catch_up
0,대신케이리츠물류1호위탁관리부동산투자회사,5. 재무제표 주석제 1(당) 기 2021년 01월 31일 현재회사명 : 주식회사...,['중요한 회계처리방침당사의 재무제표는 일반기업회계기준에 따라 작성되었으며 재무제표...,"['연법인세자산', '운영기간', '유형자산', '유동성관리규정', '연법인세자산'...",0,임정민,재무제표 확인중,부적정,2017,
1,이지스밸류플러스리츠,5. 재무제표 주석1. 회사의 개요주식회사 이지스밸류플러스위탁관리부동산투자회사(이하...,['또한 회사는 2020년 7월 16일 한국거래소 유가증권 시장에 상장을 하였으며 ...,"['신용평가기관', '연결종속기업투자주식', '유효자율금융자산', '신용한도금액',...",3,임정민,재무제표 확인중,부적정,2010,
2,JTC,5. 재무제표 주석제 27기 2021년 2월 28일 현재제 26기 2020년 2월 ...,['또한 시장을 비롯한 영업환경이 중대한 불확실성에 노출되므로 인해 고객들의 지출 ...,"['총현금유출액', '부채듀레션', '점포사용목적', '측정원칙', '경제환경', ...",0,윤채원,담당자 변경 알림: 감사A팀 최낙현 → 감사B팀 김명헌,한정,2010,
3,이씨에스,5. 재무제표 주석당기: 2020년 4월 1일부터 2021년 3월 31일까지전기: ...,"['한국채택국제회계기준은 국제회계기준위원회(""IASB"")가 발표한 기준서와 해석서 ...","['가중평균유통보통주식수당기', '법인세비용차감전순익', '가중평균유통보통주식수',...",3,최낙현,재무제표 확인중,의견거절,2016,
4,대신정보통신,5. 재무제표 주석제34기 2021년 3월 31일 현재제33기 2020년 3월 31...,['보고기간말 현재 경영진의 최선의 판단을 기준으로 한 추정치와 가정이 실제 환경과...,"['가중평균유통보통주식수당기', '당기손익인식금융부채금융부채', '비세항목손금불인정...",3,윤채원,재무제표 확인중,한정,2010,
5,기신정기,5. 재무제표 주석주석제33기 2021년 3월 31일 현재제32기 2020년 3월 ...,['재무제표는 다음을 제외하고는 역사적 원가에 기초하여 작성하였습니다.-특정 금융자...,"['법인세비용차감전순손익', '연법인세자산', '보통주당기순익', '종속기업투자주식...",3,임정민,재무제표 확인중,한정,2019,
6,이지케어텍,5. 재무제표 주석제21기 2020년 04월 01일부터 2021년 03월 31일까지...,"['당기말과 전기말 현재 회사의 주요 주주현황은 다음과 같습니다.2.', '한국채택...","['희석주당순손익희석주당손익', '가중평균유통보통주식수', '경제환경', '채무상품...",3,최낙현,재무제표 확인중,의견거절,2012,
7,신한알파리츠,5. 재무제표 주석제6 (당)기 2020년 10월 1일부터 2021년 3월 31일까...,['에 따른 별도재무제표로서 지배기업 관계기업의 투자자 또는 공동기업의 참여자가 투...,"['신용평가기관', '경영환경', '영업수익당기', '금융상품당기말', '연법인세자...",3,윤채원,재무제표 확인중,적정,2011,
8,리더스 기술투자,"5. 재무제표 주석1. 일반사항\r주식회사 리더스기술투자(이하 ""당사"" 라 함)는 ...",['해당 기준서의 개정이 재무제표에 미치는 중요한 영향은 없습니다.2.2.2 당사가...,"['희석주당순손익당기', '확정기제도확정기제도', '희석주당순손익', '손상금융자산...",1,윤채원,재무제표 확인중,적정,2010,
9,신영증권,5. 재무제표 주석1. 회사의 개요\r\r신영증권주식회사(이하\n회\n)는 금융투자...,['금융자산은 최초인식시 공정가치로 측정하고 있으며 당기손익-공정가치금융자산이 아닌...,"['상각후원가측정금융자산', '상각후원가측정금융자산', '거래일손익인식평가조정', ...",2,서수빈,재무제표 확인중,부적정,2015,


In [154]:
# 기업 정보와 catch-up messages 병합 

corp_db["catch_up"] =corp_db.merge(catch_up_fin,
                          left_on = ['corp_name'],
                          right_on = ['corp'],
                          how = "left")["message"]

In [155]:
# 병합 확인 

corp_db

,corp_name,Busines_report,summary,kye_list,classify,manager,recent_progress,last_audit grade,corp_size,catch_up
0,대신케이리츠물류1호위탁관리부동산투자회사,5. 재무제표 주석제 1(당) 기 2021년 01월 31일 현재회사명 : 주식회사...,['중요한 회계처리방침당사의 재무제표는 일반기업회계기준에 따라 작성되었으며 재무제표...,"['연법인세자산', '운영기간', '유형자산', '유동성관리규정', '연법인세자산'...",0,임정민,재무제표 확인중,부적정,2017,"세부원장 확인중, 재고실사 해당없음, 금융기관조회서 일치확인, 감사의견 적정"
1,이지스밸류플러스리츠,5. 재무제표 주석1. 회사의 개요주식회사 이지스밸류플러스위탁관리부동산투자회사(이하...,['또한 회사는 2020년 7월 16일 한국거래소 유가증권 시장에 상장을 하였으며 ...,"['신용평가기관', '연결종속기업투자주식', '유효자율금융자산', '신용한도금액',...",3,임정민,재무제표 확인중,부적정,2010,"부속명세서, 세부원장 확인중, 재고실사 해당없음"
2,JTC,5. 재무제표 주석제 27기 2021년 2월 28일 현재제 26기 2020년 2월 ...,['또한 시장을 비롯한 영업환경이 중대한 불확실성에 노출되므로 인해 고객들의 지출 ...,"['총현금유출액', '부채듀레션', '점포사용목적', '측정원칙', '경제환경', ...",0,윤채원,담당자 변경 알림: 감사A팀 최낙현 → 감사B팀 김명헌,한정,2010,감사의견 피드백 : 재검토 요망
3,이씨에스,5. 재무제표 주석당기: 2020년 4월 1일부터 2021년 3월 31일까지전기: ...,"['한국채택국제회계기준은 국제회계기준위원회(""IASB"")가 발표한 기준서와 해석서 ...","['가중평균유통보통주식수당기', '법인세비용차감전순익', '가중평균유통보통주식수',...",3,최낙현,재무제표 확인중,의견거절,2016,감사증거(회계기록) 추가 확보를 위한 이씨에스 출장 필요
4,대신정보통신,5. 재무제표 주석제34기 2021년 3월 31일 현재제33기 2020년 3월 31...,['보고기간말 현재 경영진의 최선의 판단을 기준으로 한 추정치와 가정이 실제 환경과...,"['가중평균유통보통주식수당기', '당기손익인식금융부채금융부채', '비세항목손금불인정...",3,윤채원,재무제표 확인중,한정,2010,대신정보통신 담당자 변경 알림: 감사A팀 백지원 → 감사B팀 김명헌
5,기신정기,5. 재무제표 주석주석제33기 2021년 3월 31일 현재제32기 2020년 3월 ...,['재무제표는 다음을 제외하고는 역사적 원가에 기초하여 작성하였습니다.-특정 금융자...,"['법인세비용차감전순손익', '연법인세자산', '보통주당기순익', '종속기업투자주식...",3,임정민,재무제표 확인중,한정,2019,NaN
6,이지케어텍,5. 재무제표 주석제21기 2020년 04월 01일부터 2021년 03월 31일까지...,"['당기말과 전기말 현재 회사의 주요 주주현황은 다음과 같습니다.2.', '한국채택...","['희석주당순손익희석주당손익', '가중평균유통보통주식수', '경제환경', '채무상품...",3,최낙현,재무제표 확인중,의견거절,2012,NaN
7,신한알파리츠,5. 재무제표 주석제6 (당)기 2020년 10월 1일부터 2021년 3월 31일까...,['에 따른 별도재무제표로서 지배기업 관계기업의 투자자 또는 공동기업의 참여자가 투...,"['신용평가기관', '경영환경', '영업수익당기', '금융상품당기말', '연법인세자...",3,윤채원,재무제표 확인중,적정,2011,NaN
8,리더스 기술투자,"5. 재무제표 주석1. 일반사항\r주식회사 리더스기술투자(이하 ""당사"" 라 함)는 ...",['해당 기준서의 개정이 재무제표에 미치는 중요한 영향은 없습니다.2.2.2 당사가...,"['희석주당순손익당기', '확정기제도확정기제도', '희석주당순손익', '손상금융자산...",1,윤채원,재무제표 확인중,적정,2010,NaN
9,신영증권,5. 재무제표 주석1. 회사의 개요\r\r신영증권주식회사(이하\n회\n)는 금융투자...,['금융자산은 최초인식시 공정가치로 측정하고 있으며 당기손익-공정가치금융자산이 아닌...,"['상각후원가측정금융자산', '상각후원가측정금융자산', '거래일손익인식평가조정', ...",2,서수빈,재무제표 확인중,부적정,2015,NaN


In [156]:
# 회사 별 확인 
corp_name = "JTC" #회사명을 입력해주세요 

corp_db[corp_db['corp_name'].str.contains(corp_name)]

,corp_name,Busines_report,summary,kye_list,classify,manager,recent_progress,last_audit grade,corp_size,catch_up
2,JTC,5. 재무제표 주석제 27기 2021년 2월 28일 현재제 26기 2020년 2월 ...,['또한 시장을 비롯한 영업환경이 중대한 불확실성에 노출되므로 인해 고객들의 지출 ...,"['총현금유출액', '부채듀레션', '점포사용목적', '측정원칙', '경제환경', ...",0,윤채원,담당자 변경 알림: 감사A팀 최낙현 → 감사B팀 김명헌,한정,2010,감사의견 피드백 : 재검토 요망
